In [4]:
import os 
import sys
import tensorflow as tf
import numpy as np
import pickle
from tensorflow.keras.models import load_model

sys.path.append("../")

from src.data.make_dataset import INDEX_SHEET_NAME,load_data
from src.features.build_features import generate_features
from src.models.LSTM import build_lstm_model, generate_train_val_data
from src.models.metrics import LinearCorrelation, MeanAbsolutePercentageError, TheilU

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [5]:
INDEX_SHEET_NAME

['HangSeng Index Data',
 'S&P500 Index Data',
 'CSI300 Index Data',
 'DJIA index Data',
 'Nikkei 225 index Data',
 'Nifty 50 index Data']

In [6]:
raw = load_data(sheet_name=INDEX_SHEET_NAME[-1])
raw.head()

,Ntime,Time,Closing Price,Open Price,High price,Low Price,Volume,MACD,CCI,ATR,...,EMA20,MA10,MTM6,MA5,MTM12,ROC,SMI,WVAD,US Dollar Index,HIBOR
0,20080702,733591,21704.4492,21785.3906,21938.1992,21555.5293,254858.26,-646.053200,-193.115118,546.4805,...,22956.709411,22497.96286,-751.5703,22187.92768,-887.8516,-5.754486,-0.069509,-401619.369837,71.99,1.6
1,20080703,733592,21242.7793,21389.4902,21742.0703,21163.5703,272528.48,-702.282620,-186.955957,578.5000,...,22793.477972,22289.66071,-1392.3806,21909.45156,-1786.9102,-7.872373,-0.068201,-507305.257079,72.73,1.6
2,20080704,733593,21423.8203,21402.1699,21534.0508,21344.8496,179634.42,-723.891770,-133.742667,291.2715,...,22663.034384,22152.28180,-1031.8496,21703.08164,-1634.1699,-8.153977,-0.058111,-398848.614015,72.71,1.6
3,20080707,733596,21913.0605,21402.6992,21916.2109,21402.6992,196457.92,-693.544830,-77.445777,513.5117,...,22591.608300,22069.02789,-129.2891,21677.22382,-1412.7403,-3.880007,-0.035303,-264228.169678,72.71,1.6
4,20080708,733597,21220.8105,21632.6992,21684.2109,21098.8398,201519.68,-717.087314,-108.634123,814.2207,...,22461.056128,21919.61285,-881.1993,21500.98396,-1576.7989,-6.703666,-0.055006,-518512.844172,72.96,1.6


### WSAE-LSTM

In [ ]:
result_dict = dict()
EPOCHS = 500 #0
past_history = 4

for index in INDEX_SHEET_NAME[1:]:
    print(f"Start {index} part!")
    result_dict[index] = dict()

    data_dir = f'../data/processed/wsae/{index}'
    if not os.path.exists(data_dir):
        raw = load_data(sheet_name=index)
        generate_features(raw, index)
    
    train_lst = os.listdir(data_dir)
    train_lst.sort()
    for name in train_lst:
        x_train = np.load(data_dir + f'/{name}/X_train.npy')
        y_train = np.load(data_dir + f'/{name}/Y_train.npy')
        x_val = np.load(data_dir + f'/{name}/X_val.npy')
        y_val = np.load(data_dir + f'/{name}/Y_val.npy')
        x_test = np.load(data_dir + f'/{name}/X_test.npy')
        y_test = np.load(data_dir + f'/{name}/Y_test.npy')

        train_data, val_data, test_data = generate_train_val_data(
            x_train, y_train, x_val, y_val, x_test, y_test,
            past_history=4, batch_size=60
        )


        model_save_dir = f'../models/{index}/{name}'
        if not os.path.exists(model_save_dir):
            os.makedirs(model_save_dir)
        if not os.path.exists(model_save_dir+'/wase-lstmt_config.json') \
            or not os.path.exists(model_save_dir+'wase-lstm_weights.h5'):
            print("No existing model, start to train!")
            lstm = build_lstm_model(inputs_shape=[4, 10],
                                    layers=5,
                                    units=[64, 64, 64, 64, 64],
                                    learning_rate=0.05)
            lstm.fit(train_data,
                     epochs=EPOCHS,
                     steps_per_epoch=(y_train.shape[0] // 60),
                     validation_data=val_data,
                     validation_steps=1,
                     verbose=0)
            json_config = lstm.to_json()

            with open(model_save_dir+'/wase-lstmt_config.json', 'w') as json_file:
                json_file.write(json_config)
            # Save weights to disk
            lstm.save_weights(model_save_dir+'/wase-lstm_weights.h5')
            print("Model Saved!")
        else:
            with open(model_save_dir+'/wase-lstmt_config.json') as json_file:
                json_config = json_file.read()
            lstm = tf.keras.models.model_from_json(json_config)
            lstm.compile(loss='mse',
                              optimizer='Adam',
                              metrics=[
                                  tf.keras.metrics.MeanAbsolutePercentageError(),
                                  MeanAbsolutePercentageError(),
                                  LinearCorrelation(),
                                  TheilU()],
                              lr=0.05
                              )
            lstm.load_weights(model_save_dir+'wase-lstm_weights.h5')

            print("Model Loaded!")

        result_dict[index][name] = lstm.evaluate(test_data, steps=1)

        print(f">>>>{index} {name} done!<<<<")
    try:
        with open(f'./WSAE-LSTM/{index}_train_result.pickle', 'wb') as handle:
            pickle.dump(result_dict[index], handle, protocol=pickle.HIGHEST_PROTOCOL)
    except:
        print('fail to save!')

with open(f'./WSAE-LSTM/train_result.pickle', 'wb') as handle:
    pickle.dump(result_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

Start S&P500 Index Data part!
>>>>201010 finished!<<<<
>>>>201101 finished!<<<<
>>>>201104 finished!<<<<
>>>>201107 finished!<<<<
>>>>201110 finished!<<<<
>>>>201201 finished!<<<<
>>>>201204 finished!<<<<
>>>>201207 finished!<<<<
>>>>201210 finished!<<<<
>>>>201301 finished!<<<<
>>>>201304 finished!<<<<
>>>>201307 finished!<<<<
>>>>201310 finished!<<<<
>>>>201401 finished!<<<<
>>>>201404 finished!<<<<
>>>>201407 finished!<<<<
>>>>201410 finished!<<<<
>>>>201501 finished!<<<<
>>>>201504 finished!<<<<
>>>>201507 finished!<<<<
>>>>201510 finished!<<<<
>>>>201601 finished!<<<<
>>>>201604 finished!<<<<
>>>>201607 finished!<<<<
>>>> Feature generation complete! <<<<
No existing model, start to train!
Model Saved!
1/1 [==============================] - 0s 11ms/step - loss: 5.8790e-04 - mean_absolute_percentage_error: 2.2837 - mape: 0.0228 - r: 0.8632 - theil_u: 0.0143
>>>>S&P500 Index Data 201010 done!<<<<
No existing model, start to train!
Model Saved!
1/1 [==============================] - 

In [ ]:
import pickle
with open(f'train_result.pickle', 'rb') as handle:
    result_dict=pickle.load(handle)

In [11]:
raw=load_data(sheet_name='CSI300 Index Data')

In [12]:
raw.head()

,Ntime,Time,Closing Price,Open Price,High price,Low Price,Volume,MACD,CCI,ATR,...,EMA20,MA10,MTM6,MA5,MTM12,ROC,SMI,WVAD,US Dollar Index,HIBOR
0,20080702,733591,21704.4492,21785.3906,21938.1992,21555.5293,254858.26,-646.053200,-193.115118,546.4805,...,22956.709411,22497.96286,-751.5703,22187.92768,-887.8516,-5.754486,-0.069509,-401619.369837,71.99,1.6
1,20080703,733592,21242.7793,21389.4902,21742.0703,21163.5703,272528.48,-702.282620,-186.955957,578.5000,...,22793.477972,22289.66071,-1392.3806,21909.45156,-1786.9102,-7.872373,-0.068201,-507305.257079,72.73,1.6
2,20080704,733593,21423.8203,21402.1699,21534.0508,21344.8496,179634.42,-723.891770,-133.742667,291.2715,...,22663.034384,22152.28180,-1031.8496,21703.08164,-1634.1699,-8.153977,-0.058111,-398848.614015,72.71,1.6
3,20080707,733596,21913.0605,21402.6992,21916.2109,21402.6992,196457.92,-693.544830,-77.445777,513.5117,...,22591.608300,22069.02789,-129.2891,21677.22382,-1412.7403,-3.880007,-0.035303,-264228.169678,72.71,1.6
4,20080708,733597,21220.8105,21632.6992,21684.2109,21098.8398,201519.68,-717.087314,-108.634123,814.2207,...,22461.056128,21919.61285,-881.1993,21500.98396,-1576.7989,-6.703666,-0.055006,-518512.844172,72.96,1.6


In [ ]:
result_dict

### WLSTM

In [ ]:
result_dict = dict()
EPOCHS = 500 #0
past_history = 4

for index in INDEX_SHEET_NAME:
    print(f"Start {index} part!")
    result_dict[index] = dict()

    data_dir = f'../data/processed/wavelet/{index}'
    if not os.path.exists(data_dir):
        raw = load_data(sheet_name=index)
        generate_features(raw, index)
    
    train_lst = os.listdir(data_dir)
    for name in train_lst:
        x_train = np.load(data_dir + f'/{name}/X_train.npy')
        y_train = np.load(data_dir + f'/{name}/Y_train.npy')
        x_val = np.load(data_dir + f'/{name}/X_val.npy')
        y_val = np.load(data_dir + f'/{name}/Y_val.npy')
        x_test = np.load(data_dir + f'/{name}/X_test.npy')
        y_test = np.load(data_dir + f'/{name}/Y_test.npy')

        train_data, val_data, test_data = generate_train_val_data(
            x_train, y_train, x_val, y_val, x_test, y_test,
            past_history=4, batch_size=60
        )


        model_save_dir = f'../models/{index}/{name}'
        config_filename = '/wlstmt_config.json'
        weight_filename = 'wlstm_weights.h5'

        if not os.path.exists(model_save_dir):
            os.makedirs(model_save_dir)
        if not os.path.exists(model_save_dir+config_filename) \
            or not os.path.exists(model_save_dir+weight_filename):
            print("No existing model, start to train!")
            lstm = build_lstm_model(inputs_shape=[4, 10],
                                    layers=5,
                                    units=[64, 64, 64, 64, 64],
                                    learning_rate=0.05)
            lstm.fit(train_data,
                     epochs=EPOCHS,
                     steps_per_epoch=(y_train.shape[0] // 60),
                     validation_data=val_data,
                     validation_steps=1,
                     verbose=0)
            json_config = lstm.to_json()

            with open(model_save_dir+config_filename, 'w') as json_file:
                json_file.write(json_config)
            # Save weights to disk
            lstm.save_weights(model_save_dir+weight_filename)
            print("Model Saved!")
        else:
            with open(model_save_dir+config_filename) as json_file:
                json_config = json_file.read()
            lstm = tf.keras.models.model_from_json(json_config)
            lstm.compile(loss='mse',
                              optimizer='Adam',
                              metrics=[
                                  tf.keras.metrics.MeanAbsolutePercentageError(),
                                  MeanAbsolutePercentageError(),
                                  LinearCorrelation(),
                                  TheilU()],
                              lr=learning_rate
                              )
            lstm.load_weights(model_save_dir+weight_filename)

            print("Model Loaded!")

        result_dict[index][name] = lstm.evaluate(test_data, steps=1)

        print(f">>>>{index} {name} done!<<<<")
    try:
        with open(f'./WLSTM/{index}_train_result.pickle', 'wb') as handle:
            pickle.dump(result_dict[index], handle, protocol=pickle.HIGHEST_PROTOCOL)
    except:
        print('fail to save!')

with open(f'./WLSTM/train_result.pickle', 'wb') as handle:
    pickle.dump(result_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)